In [2]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv1D,MaxPooling1D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
matplotlib.use('agg')
import pylab as plt
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import Normalizer
from keras.utils import np_utils
import pickle
import h5py


# KDD - and - 
X_testN_ = pd.read_pickle("./dataset/kdd_test__2labels.pkl")
X_trainN_ = pd.read_pickle("./dataset/kdd_train__2labels.pkl")
Y_trainN_ = X_trainN_.iloc[:, 122:].as_matrix()
Y_testN_ = X_testN_.iloc[:, 122:].as_matrix()
X_trainN_ = X_trainN_.as_matrix()
X_testN_ = X_testN_.as_matrix()
scaler1 = Normalizer().fit(X_trainN_[:,:122])
X_trainN_ = scaler1.transform(X_trainN_[:,:122])
scaler2 = Normalizer().fit(X_testN_[:,:122])
X_testN_ = scaler2.transform(X_testN_[:,:122])
X_trainN_ = np.reshape(X_trainN_, (X_trainN_.shape[0],X_trainN_.shape[1],1))
X_testN_ = np.reshape(X_testN_, (X_testN_.shape[0],X_testN_.shape[1],1))
print(Y_testN_.shape)
print(X_testN_.shape)
print(Y_trainN_.shape)
print(X_trainN_.shape)

(11850, 2)
(11850, 122, 1)
(25192, 2)
(25192, 122, 1)


In [3]:
cnn1d_1 = Sequential([Conv1D(64, 3, padding="same",input_shape=(122, 1)),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(2),
    Activation('sigmoid'),
])
print(cnn1d_1.summary())

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 122, 64)           256       
_________________________________________________________________
activation_1 (Activation)    (None, 122, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 61, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3904)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               499840    
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
___________________________________________________________

In [4]:
#cnn1d_1 = multi_gpu_model(cnn1d_1, gpus=8)
cnn1d_1.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
history = cnn1d_1.fit(X_trainN_, Y_trainN_, batch_size=64, validation_data=(X_testN_, Y_testN_) ,epochs=25)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn1d_1.to_json()
with open("NN.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn1d_1.save_weights("NN.h5")
print("Saved model to disk")
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Acc_ValaccuaracyNN.png')

Train on 25192 samples, validate on 11850 samples
Epoch 1/25
25192/25192 [==============================] - 7s 287us/step - loss: 0.2231 - acc: 0.9129 - val_loss: 1.6157 - val_acc: 0.4208
Epoch 2/25
25192/25192 [==============================] - 7s 263us/step - loss: 0.1496 - acc: 0.9448 - val_loss: 1.7222 - val_acc: 0.4970
Epoch 3/25
25192/25192 [==============================] - 7s 264us/step - loss: 0.1304 - acc: 0.9536 - val_loss: 1.7017 - val_acc: 0.5281
Epoch 4/25
25192/25192 [==============================] - 7s 287us/step - loss: 0.1217 - acc: 0.9572 - val_loss: 1.5662 - val_acc: 0.5559
Epoch 5/25
25192/25192 [==============================] - 6s 255us/step - loss: 0.1119 - acc: 0.9611 - val_loss: 1.8772 - val_acc: 0.5482
Epoch 6/25
25192/25192 [==============================] - 6s 246us/step - loss: 0.1077 - acc: 0.9611 - val_loss: 1.7734 - val_acc: 0.5570
Epoch 7/25
25192/25192 [==============================] - 6s 244us/step - loss: 0.1033 - acc: 0.9639 - val_loss: 1.9736 - 